In [2]:
!conda install cv2 -y

^C


In [5]:
import pygame
import cv2
import numpy as np
import math

# Initialize Pygame
pygame.init()

# Screen Dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Autonomous Vehicle Simulation")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Car Attributes
car_width, car_height = 50, 30
car_x, car_y = WIDTH // 2, HEIGHT - 100
car_speed = 0
car_angle = 0

# Lane Detection Settings
lane_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
lane_center = WIDTH // 2

# Obstacles
obstacles = [pygame.Rect(300, 400, 50, 50), pygame.Rect(500, 300, 50, 50)]

# Clock for frame rate
clock = pygame.time.Clock()

def detect_lanes(image):
    """Simulates lane detection using OpenCV."""
    global lane_center

    # Create a blank road with lanes
    road = np.zeros_like(image)
    cv2.line(road, (WIDTH // 3, 0), (WIDTH // 3, HEIGHT), (255, 255, 255), 2)
    cv2.line(road, (2 * WIDTH // 3, 0), (2 * WIDTH // 3, HEIGHT), (255, 255, 255), 2)
    blended = cv2.addWeighted(image, 0.8, road, 1, 0)

    # Lane center is simulated as the middle of the road
    lane_center = WIDTH // 2

    return blended

def draw_car(x, y, angle):
    """Draws the car with the given position and angle."""
    car_surface = pygame.Surface((car_width, car_height), pygame.SRCALPHA)
    car_surface.fill(BLUE)
    rotated_car = pygame.transform.rotate(car_surface, angle)
    rect = rotated_car.get_rect(center=(x, y))
    screen.blit(rotated_car, rect.topleft)

def autonomous_control():
    """Controls the car autonomously to stay in the lane and avoid obstacles."""
    global car_x, car_y, car_speed, car_angle

    # Adjust car's position to align with the lane center
    if car_x < lane_center - 10:
        car_angle -= 1  # Turn right
    elif car_x > lane_center + 10:
        car_angle += 1  # Turn left
    else:
        car_angle = 0  # Straight

    # Move the car forward
    car_speed = 2
    car_x += car_speed * math.cos(math.radians(car_angle))
    car_y -= car_speed * math.sin(math.radians(car_angle))

    # Obstacle avoidance
    for obstacle in obstacles:
        if pygame.Rect(car_x - car_width // 2, car_y - car_height // 2, car_width, car_height).colliderect(obstacle):
            car_speed = 0  # Stop on collision

# Main Loop
running = True
while running:
    screen.fill(BLACK)

    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Simulate lane detection using OpenCV
    lane_image = detect_lanes(lane_image)

    # Convert OpenCV image to Pygame surface
    lane_surface = pygame.surfarray.make_surface(cv2.cvtColor(lane_image, cv2.COLOR_BGR2RGB))
    screen.blit(lane_surface, (0, 0))

    # Draw obstacles
    for obstacle in obstacles:
        pygame.draw.rect(screen, RED, obstacle)

    # Autonomous control and car movement
    autonomous_control()
    draw_car(car_x, car_y, car_angle)

    # Update the display
    pygame.display.flip()
    clock.tick(30)

pygame.quit()


KeyboardInterrupt: 